In [27]:
import pandas as pd
import numpy as np
import math
import chart_studio.plotly as py
import plotly.tools as tls
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, accuracy_score, log_loss, mean_squared_error, f1_score



In [28]:
# Load model from file
xnn_dir = '/home/kimm/article-information-2019/data/xnn_output/hmda_results/'

filename = 'hmda_ann_results_with_Shapley.csv'

TEST = pd.read_csv(xnn_dir + filename)

Feature_names = ['term_360', 'conforming', 'debt_to_income_ratio_missing','loan_amount_std', 
                 'loan_to_value_ratio_std', 'no_intro_rate_period_std', 'intro_rate_period_std',
                 'property_value_std', 'income_std', 'debt_to_income_ratio_std']

In [29]:
TEST.columns

Index(['Unnamed: 0', 'high_priced', 'term_360', 'conforming', 'black', 'asian',
       'white', 'amind', 'hipac', 'hispanic', 'non_hispanic', 'male', 'female',
       'agegte62', 'agelt62', 'debt_to_income_ratio_missing',
       'loan_amount_std', 'loan_to_value_ratio_std',
       'no_intro_rate_period_std', 'intro_rate_period_std',
       'property_value_std', 'income_std', 'debt_to_income_ratio_std',
       'probability', 'term_360_Shapley_score', 'conforming_Shapley_score',
       'debt_to_income_ratio_missing_Shapley_score',
       'loan_amount_std_Shapley_score',
       'loan_to_value_ratio_std_Shapley_score',
       'no_intro_rate_period_std_Shapley_score',
       'intro_rate_period_std_Shapley_score',
       'property_value_std_Shapley_score', 'income_std_Shapley_score',
       'debt_to_income_ratio_std_Shapley_score', 'Intercept_Shapley_score'],
      dtype='object')

In [30]:
def get_prauc(frame, y, yhat, pos=1, neg=0, res=0.01):
    
    """ Calculates precision, recall, and f1 for a pandas dataframe of y 
        and yhat values.
    
    Args:
        frame: Pandas dataframe of actual (y) and predicted (yhat) values.
        y: Name of actual value column.
        yhat: Name of predicted value column.
        pos: Primary target value, default 1.
        neg: Secondary target value, default 0.
        res: Resolution by which to loop through cutoffs, default 0.01.
    
    Returns:
        Pandas dataframe of precision, recall, and f1 values.
        
    """
    
    frame_ = frame.copy(deep=True) # don't destroy original data
    dname = 'd_' + str(y)          # column for predicted decisions
    eps = 1e-20                    # for safe numerical operations
    
    # init p-r roc frame
    prroc_frame = pd.DataFrame(columns=['cutoff', 'recall', 'precision', 'f1'])
    
    # loop through cutoffs to create p-r roc frame
    for cutoff in np.arange(0, 1 + res, res):

        # binarize decision to create confusion matrix values
        frame_[dname] = np.where(frame_[yhat] > cutoff , 1, 0)
        
        # calculate confusion matrix values
        tp = frame_[(frame_[dname] == pos) & (frame_[y] == pos)].shape[0]
        fp = frame_[(frame_[dname] == pos) & (frame_[y] == neg)].shape[0]
        tn = frame_[(frame_[dname] == neg) & (frame_[y] == neg)].shape[0]
        fn = frame_[(frame_[dname] == neg) & (frame_[y] == pos)].shape[0]

        # calculate precision, recall, and f1
        recall = (tp + eps)/((tp + fn) + eps)
        precision = (tp + eps)/((tp + fp) + eps)
        f1 = 2/((1/(recall + eps)) + (1/(precision + eps)))
        
        # add new values to frame
        prroc_frame = prroc_frame.append({'cutoff': cutoff,
                                          'recall': recall,
                                          'precision': precision,
                                          'f1': f1}, 
                                          ignore_index=True)
    
    # housekeeping
    del frame_
    
    return prroc_frame

# calculate and display recall and precision
#prauc_frame = get_prauc(test_yhat, y, yhat)
prauc_frame = get_prauc(TEST, 'high_priced', 'probability')
prauc_frame.style.set_caption('Recall and Precision')

,cutoff,recall,precision,f1
0,0,0.999741,0.0976712,0.177957
1,0.01,0.981903,0.143132,0.249844
2,0.02,0.963547,0.170713,0.290039
3,0.03,0.947001,0.19111,0.318038
4,0.04,0.926319,0.207891,0.339573
5,0.05,0.90667,0.223475,0.358571
6,0.06,0.889607,0.239407,0.377282
7,0.07,0.866081,0.252849,0.391424
8,0.08,0.847208,0.265215,0.403969
9,0.09,0.826008,0.277561,0.415502


In [31]:
xnn_cut = prauc_frame.loc[prauc_frame['f1'].idxmax(), 'cutoff'] # value associated w/ index of max. F1
print('Best F1 threshold: %.2f' % xnn_cut)

Best F1 threshold: 0.22


In [32]:
# Calculate test statistics
Prediction = list(TEST['probability'])
Classification = list(TEST['probability'].apply(lambda x: int(x >= 0.22)))
Actual = list(TEST['high_priced'].apply(int))

test_statistics = {}

test_statistics['AUC'] = roc_auc_score(Actual, Prediction)
test_statistics['accuracy_score'] = accuracy_score(Actual, Classification)
test_statistics['log_loss'] = log_loss(Actual, Prediction)
test_statistics['rmse'] = math.sqrt(mean_squared_error(Actual, Prediction))

print(test_statistics)

{'AUC': 0.871052663092998, 'rmse': 0.2623143268286164, 'accuracy_score': 0.8649589027280521, 'log_loss': 0.23114619380565204}
